<a href="https://colab.research.google.com/github/mab2004/histopathological-cancer-detection/blob/main/histopathological_cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upload the zipped folder
from google.colab import files

print("Please upload your 'cancer_detection.zip' file.")
uploaded = files.upload()

Please upload your 'cancer_detection.zip' file.


Saving cancer_detection.zip to cancer_detection.zip


In [ ]:
# Unzip the folder
!unzip cancer_detection.zip

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your dataset directories in Colab
train_dir = 'cancer_detection/train/'
test_dir = 'cancer_detection/test/'

# Set image parameters
image_size = (128, 128)
batch_size = 32

# Create a data generator for training and validation data.
# 'rescale' handles pixel normalization.
# 'validation_split' reserves 20% of the training data for validation.
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create a data generator for the test set.
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data from the specified directory.
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Load the validation data from the same directory.
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Load the test data from the test directory.
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

print("\nData generators have been set up and images loaded.")
print(f"Number of training images: {train_generator.n}")
print(f"Number of validation images: {validation_generator.n}")
print(f"Number of test images: {test_generator.n}")

Found 7231 images belonging to 2 classes.
Found 1806 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.

Data generators have been set up and images loaded.
Number of training images: 7231
Number of validation images: 1806
Number of test images: 1000


In [4]:
# Build the CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Initialize the model
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Third Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the feature maps
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Display the model architecture
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Model has been compiled and is ready for training.")

Model has been compiled and is ready for training.


In [7]:
# Train the Model (Corrected Code)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.8652 - loss: 0.3225 - val_accuracy: 0.8068 - val_loss: 0.5257
Epoch 2/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.8749 - loss: 0.3067 - val_accuracy: 0.8090 - val_loss: 0.5505
Epoch 3/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.8902 - loss: 0.2729 - val_accuracy: 0.7918 - val_loss: 0.5270
Epoch 4/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.8925 - loss: 0.2699 - val_accuracy: 0.8001 - val_loss: 0.5584
Epoch 5/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.8970 - loss: 0.2505 - val_accuracy: 0.7990 - val_loss: 0.5790
Epoch 6/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9255 - loss: 0.1878 - val_accuracy: 0.7641 - val_loss: 0.7562
Epoch 7/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9478 - loss: 0.1456 - val_accuracy: 0.7381 - val_loss: 0.8061
Epoch 8/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9553 - loss: 0.1234 - val_acc

In [8]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)

print("\n---------------------------------")
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.2f}")
print("---------------------------------")

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.7007 - loss: 1.7485

---------------------------------
Test Loss: 1.7059
Test Accuracy: 0.70
---------------------------------


In [9]:
# Build a Transfer Learning Model (VGG16)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

# Load the VGG16 model, pre-trained on ImageNet, without the top classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the convolutional base
# This means its weights will not be updated during training
for layer in base_model.layers:
    layer.trainable = False

# Build the new model on top of the VGG16 base
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# This is the full model we will train
model_vgg = Model(inputs=base_model.input, outputs=predictions)

# Compile the new model
model_vgg.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Display the new model architecture
model_vgg.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,763,521 (60.13 MB)

 Trainable params: 1,048,833 (4.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [10]:
# Train the Transfer Learning Model
history_vgg = model_vgg.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 29s 101ms/step - accuracy: 0.7960 - loss: 0.4673 - val_accuracy: 0.7628 - val_loss: 0.4868
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8125 - loss: 0.3463 - val_accuracy: 0.7740 - val_loss: 0.4703
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 41s 147ms/step - accuracy: 0.8686 - loss: 0.3129 - val_accuracy: 0.7706 - val_loss: 0.4845
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9062 - loss: 0.3185 - val_accuracy: 0.7656 - val_loss: 0.4914
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 21s 92ms/step - accuracy: 0.8747 - loss: 0.2899 - val_accuracy: 0.7857 - val_loss: 0.4652
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9062 - loss: 0.2375 - val_accuracy: 0.7974 - val_loss: 0.4546
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 41s 107ms/step - accuracy: 0.8893 - loss: 0.2591 - val_accuracy: 0.7796 - val_loss: 0.5228
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8438 - loss: 0.5439 - v

In [11]:
# Evaluate the Transfer Learning model on the test set
loss_vgg, accuracy_vgg = model_vgg.evaluate(test_generator)

print("\n---------------------------------")
print(f"Transfer Learning Test Loss: {loss_vgg:.4f}")
print(f"Transfer Learning Test Accuracy: {accuracy_vgg:.2f}")
print("---------------------------------")

32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - accuracy: 0.8176 - loss: 0.4429

---------------------------------
Transfer Learning Test Loss: 0.4426
Transfer Learning Test Accuracy: 0.81
---------------------------------
